In [20]:
import json

with open('./train_data/product_category_test.json', encoding='UTF8') as f:
    train = json.load(f)
with open('./train_data/product_category_valid.json', encoding='UTF8') as f:
    test = json.load(f)
len(train), len(test)

(43061, 4306)

In [21]:
import pandas as pd
from tqdm import tqdm
df_train = pd.DataFrame(columns=['document', 'label'])
for i in tqdm(range(len(train))):
    temp_list = [train[i]['title'], train[i]['category']]
    df_train.loc[ len(df_train)] = temp_list
    
df_train.head()

100%|██████████| 43061/43061 [00:31<00:00, 1364.19it/s]


,document,label
0,파바빈프로테인 파바빈가루 파바민 헬스보조제 기타단백질보충제 피토스테롤 이소플라본...,다이어트식품
1,아이티씨 퓨리웨이 RP800A 3D 펜 + PLA 필라멘트 20색 세트,주변기기
2,배터리 밧데리 연결선 50SQ,자동차용품
3,포시엘라 빨아쓰는 규조토 발매트 3세대 욕실 주방 매트 북유럽 꽃반원,카페트/러그
4,마리끌레르 장희령착용 케이블 라운드 로고 풀오버 MODBKN08M3,여성의류


In [22]:
df_test = pd.DataFrame( columns=['document', 'label'])
for i in tqdm(range(len(test))):
    temp_list = [test[i]['title'], test[i]['category']]
    df_test.loc[ len(df_test)] = temp_list
    
df_test.head()

100%|██████████| 4306/4306 [00:01<00:00, 2939.06it/s]


,document,label
0,2인용 라탄 행잉체어 스윙 그네 의자,아웃도어가구
1,올드솔 아기 신발 돌 유아 첫 걸음마 신발 아기운동화 120 130 140 밤비...,유아동잡화
2,계단 운반 끌차 상업용 가구 박스 물류 이동 접이식,공구
3,빨아쓰는 플라워 규조토 발매트,카페트/러그
4,당일조업 서해안 봄 암 꽃게 1kg 활 생물,수산물


In [23]:
from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(sentence):
    return ' '.join(okt.morphs(sentence))

df_train['tokenized_document'] = df_train['document'].apply(lambda x:okt_tokenizer(x))
df_test['tokenized_document'] = df_test['document'].apply(lambda x:okt_tokenizer(x))
df_train.head()

,document,label,tokenized_document
0,파바빈프로테인 파바빈가루 파바민 헬스보조제 기타단백질보충제 피토스테롤 이소플라본...,다이어트식품,파바빈 프로 테 인 파바빈 가루 파바민 헬스 보조 제 기타 단백질 보충 제 피토 스...
1,아이티씨 퓨리웨이 RP800A 3D 펜 + PLA 필라멘트 20색 세트,주변기기,아이티 씨 퓨리 웨이 RP 800 A 3 D 펜 + PLA 필라멘트 20 색 세트
2,배터리 밧데리 연결선 50SQ,자동차용품,배터리 밧데리 연 결선 50 SQ
3,포시엘라 빨아쓰는 규조토 발매트 3세대 욕실 주방 매트 북유럽 꽃반원,카페트/러그,포 시엘 라 빨 아 쓰는 규조토 발 매트 3 세대 욕실 주방 매트 북유럽 꽃 반원
4,마리끌레르 장희령착용 케이블 라운드 로고 풀오버 MODBKN08M3,여성의류,마리 끌 레르 장 희 령 착용 케이블 라운드 로고 풀 오버 MODBKN 08 M 3


In [24]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load('./w2v/word2vec.model')
len(w2v_model.wv.index_to_key)

22828

In [27]:
def token_to_idx(tokenized_document, unk_idx):
    idx_list = []
    for token in tokenized_document.split(' '):
        try:
            idx_list.append( w2v_model.wv.index_to_key.index(token) )
        except:
            idx_list.append(unk_idx)
    return idx_list

w2v_model.wv.index_to_key

['.',
 '이',
 '영화',
 '의',
 '..',
 '가',
 '에',
 '을',
 '...',
 '도',
 '들',
 ',',
 '는',
 '를',
 '은',
 '너무',
 '?',
 '한',
 '다',
 '정말',
 '적',
 '만',
 '!',
 '진짜',
 '으로',
 '점',
 '로',
 '에서',
 '연기',
 '과',
 '평점',
 '것',
 '최고',
 '내',
 '~',
 '나',
 '그',
 '잘',
 '와',
 '안',
 '인',
 '이런',
 '스토리',
 '생각',
 '못',
 '....',
 '왜',
 '드라마',
 '게',
 '이다',
 '감동',
 '사람',
 '1',
 '하는',
 '보고',
 '하고',
 '말',
 '고',
 '아',
 '더',
 '때',
 'ㅋㅋ',
 '배우',
 '거',
 '감독',
 '그냥',
 '요',
 '본',
 '재미',
 '내용',
 '뭐',
 '중',
 '까지',
 '!!',
 '좀',
 '쓰레기',
 '보다',
 '없는',
 '시간',
 '수',
 '지',
 '네',
 '봤는데',
 '10',
 '작품',
 '사랑',
 '할',
 '없다',
 '다시',
 '하나',
 '볼',
 '마지막',
 '이건',
 'ㅋ',
 '좋은',
 '저',
 '정도',
 '입니다',
 '같은',
 '완전',
 '있는',
 '2',
 'ㅋㅋㅋ',
 '처음',
 '장면',
 '액션',
 '주인공',
 '대',
 '걸',
 '이렇게',
 '보는',
 '개',
 'ㅠㅠ',
 '최악',
 '3',
 "'",
 '지금',
 '돈',
 '이야기',
 '별로',
 '임',
 '참',
 '하',
 '봐도',
 '느낌',
 '연출',
 '없고',
 '라',
 '끝',
 '명작',
 '듯',
 '인데',
 '별',
 '그리고',
 '역시',
 '기',
 '난',
 '많이',
 '서',
 '재밌게',
 '이해',
 '^^',
 '!!!',
 'ㅡㅡ',
 '이영화',
 '면',
 '라고',
 '보면',
 '또',
 '이나',
 '여자

In [28]:
UNK_IDX = len(w2v_model.wv.index_to_key)

df_train['doc_vectorized'] = df_train['tokenized_document'].apply(lambda x:token_to_idx(x, UNK_IDX))
df_test['doc_vectorized'] = df_test['tokenized_document'].apply(lambda x:token_to_idx(x, UNK_IDX))
df_train.head()

,document,label,tokenized_document,doc_vectorized
0,파바빈프로테인 파바빈가루 파바민 헬스보조제 기타단백질보충제 피토스테롤 이소플라본...,다이어트식품,파바빈 프로 테 인 파바빈 가루 파바민 헬스 보조 제 기타 단백질 보충 제 피토 스...,"[22828, 1043, 4161, 40, 22828, 22828, 22828, 2..."
1,아이티씨 퓨리웨이 RP800A 3D 펜 + PLA 필라멘트 20색 세트,주변기기,아이티 씨 퓨리 웨이 RP 800 A 3 D 펜 + PLA 필라멘트 20 색 세트,"[22828, 189, 15263, 6871, 22828, 22828, 2272, ..."
2,배터리 밧데리 연결선 50SQ,자동차용품,배터리 밧데리 연 결선 50 SQ,"[22828, 22828, 353, 22828, 3122, 22828]"
3,포시엘라 빨아쓰는 규조토 발매트 3세대 욕실 주방 매트 북유럽 꽃반원,카페트/러그,포 시엘 라 빨 아 쓰는 규조토 발 매트 3 세대 욕실 주방 매트 북유럽 꽃 반원,"[2789, 22828, 127, 1327, 58, 2105, 22828, 787,..."
4,마리끌레르 장희령착용 케이블 라운드 로고 풀오버 MODBKN08M3,여성의류,마리 끌 레르 장 희 령 착용 케이블 라운드 로고 풀 오버 MODBKN 08 M 3,"[2399, 1423, 22828, 690, 1724, 16256, 22828, 1..."


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 32
X_train = pad_sequences(df_train['doc_vectorized'], padding='post', truncating='post', maxlen=MAX_LEN, value= len(w2v_model.wv.index_to_key) + 1)
X_test = pad_sequences(df_test['doc_vectorized'], padding='post', truncating='post', maxlen=MAX_LEN, value= len(w2v_model.wv.index_to_key) + 1)
X_train.shape, X_test.shape

df_train['label'].unique()

array(['다이어트식품', '주변기기', '자동차용품', '카페트/러그', '여성의류', '음향가전', '공구', '화방용품',
       '악기', 'PC액세서리', '장류', 'DIY자재/용품', '태블릿PC액세서리', '바디케어', '완구/매트',
       '수납/정리용품', '식용유/오일', '물리치료/저주파용품', '주방용품', '건강식품', '축산물', '클렌징',
       '뷰티소품', '스쿼시', '스킨케어', '헤어케어', '헤어스타일링', '낚시', '농산물', '패션소품', '수영',
       '모자', '여성언더웨어/잠옷', '양말', '오토바이/스쿠터', '유가공품', '해외여행', '문구/사무용품',
       '국내여행/체험', '생활가전', '자동차기기', '반려동물', '여성신발', '생활용품', '주방가전', '음료',
       '이미용가전', '서재/사무용가구', '유모차', '주얼리', '휴대폰액세서리', '소독/살균용품', '수집품',
       'PC부품', '시계', '생활편의', '헬스', '이유식용품', '잼/시럽', '야구', '김치', '욕실용품',
       '계절가전', '과자/베이커리', '검도', '게임기/타이틀', '스포츠액세서리', '소프트웨어', '저장장치',
       '인테리어소품', '냉동/간편조리식품', '여성가방', '침구세트', '노트북', '댄스', '수산물',
       '헤어액세서리', '마스크/팩', '멀티미디어장비', '눈건강용품', '정원/원예용품', '아기간식', '지갑',
       '반찬', '캠핑', '수예', '주류', '자전거', '보호용품', '영상가전', '골프', '교구', '장갑',
       '카메라/캠코더용품', '재활운동용품', '소스/드레싱', '솜류', '라면/면류', '음반', '학습기기',
       '가루/분말류', '홈데코', '여행용가방/소품', '커튼/블라인드', '권투', '세탁용품', '침구단품',
   

In [30]:
label_list = list(df_train['label'].unique())
print(label_list)
y_train = df_train['label'].apply(lambda x : label_list.index(x))
y_test = df_test['label'].apply(lambda x : label_list.index(x))
y_train.shape, y_test.shape

y_train.unique()

['다이어트식품', '주변기기', '자동차용품', '카페트/러그', '여성의류', '음향가전', '공구', '화방용품', '악기', 'PC액세서리', '장류', 'DIY자재/용품', '태블릿PC액세서리', '바디케어', '완구/매트', '수납/정리용품', '식용유/오일', '물리치료/저주파용품', '주방용품', '건강식품', '축산물', '클렌징', '뷰티소품', '스쿼시', '스킨케어', '헤어케어', '헤어스타일링', '낚시', '농산물', '패션소품', '수영', '모자', '여성언더웨어/잠옷', '양말', '오토바이/스쿠터', '유가공품', '해외여행', '문구/사무용품', '국내여행/체험', '생활가전', '자동차기기', '반려동물', '여성신발', '생활용품', '주방가전', '음료', '이미용가전', '서재/사무용가구', '유모차', '주얼리', '휴대폰액세서리', '소독/살균용품', '수집품', 'PC부품', '시계', '생활편의', '헬스', '이유식용품', '잼/시럽', '야구', '김치', '욕실용품', '계절가전', '과자/베이커리', '검도', '게임기/타이틀', '스포츠액세서리', '소프트웨어', '저장장치', '인테리어소품', '냉동/간편조리식품', '여성가방', '침구세트', '노트북', '댄스', '수산물', '헤어액세서리', '마스크/팩', '멀티미디어장비', '눈건강용품', '정원/원예용품', '아기간식', '지갑', '반찬', '캠핑', '수예', '주류', '자전거', '보호용품', '영상가전', '골프', '교구', '장갑', '카메라/캠코더용품', '재활운동용품', '소스/드레싱', '솜류', '라면/면류', '음반', '학습기기', '가루/분말류', '홈데코', '여행용가방/소품', '커튼/블라인드', '권투', '세탁용품', '침구단품', '조미료', '네트워크장비', 'DVD', '건강관리용품', '유아동의류', '스킨/바디용품', '색조메이크업', '요가/필라테스', '구강청결용품', '임산부용품', '등산', 

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [31]:
import numpy as np
emb_matrix = np.row_stack([w2v_model.wv.vectors, np.zeros([2,100])])
emb_matrix.shape
type(emb_matrix)
emb_matrix

array([[-0.14136039,  0.21440038,  0.24057744, ..., -0.09436294,
        -0.04935567,  0.31391302],
       [ 0.23245548,  0.13568544,  0.10917881, ..., -0.07305077,
        -0.14869006, -0.03994658],
       [-0.18210478, -0.00554782, -0.1645893 , ..., -0.15364173,
         0.21794109, -0.23150666],
       ...,
       [-0.03680937,  0.20403486,  0.13213195, ..., -0.13196641,
        -0.13569278, -0.01072323],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [39]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant
import tensorflow as tf

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=5)

x = Input((MAX_LEN,))

emb = Embedding(emb_matrix.shape[0], emb_matrix.shape[1], input_shape = (MAX_LEN,), embeddings_initializer = Constant(emb_matrix))(x)
lstm = LSTM(128, activation= 'tanh', dropout = 0.2, return_sequences= False)(emb)

output = Dense(206, activation= 'softmax')(lstm)

model = tf.keras.Model(x, output)

model.summary()

with tf.device("/gpu:3"):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, batch_size =512, epochs=30, validation_data=(X_test, y_test), callbacks=[earlystop_callback])

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_8 (Embedding)         │ (None, 32, 100)        │     2,283,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 206)            │        26,574 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,426,822 (9.26 MB)

 Trainable params: 2,426,822 (9.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.0300 - loss: 4.9608 - val_accuracy: 0.0406 - val_loss: 4.7292
Epoch 2/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.0371 - loss: 4.7169 - val_accuracy: 0.0351 - val_loss: 4.7254
Epoch 3/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.0385 - loss: 4.7254 - val_accuracy: 0.0397 - val_loss: 4.7243
Epoch 4/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.0387 - loss: 4.7154 - val_accuracy: 0.0406 - val_loss: 4.7262
Epoch 5/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.0398 - loss: 4.7137 - val_accuracy: 0.0406 - val_loss: 4.7193
Epoch 6/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.0444 - loss: 4.6992 - val_accuracy: 0.0555 - val_loss: 4.5900
Epoch 7/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.0683 - loss: 4.5004 - val_accuracy: 0.0780 - val_loss: 4.3738
Epoch 8/30
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - accuracy: 0.0832 - loss: 4.3399 - val_accuracy: 0.0864 - v

In [43]:
def sentence_to_sequence(sentence):
    tokenized_sentence = ' '.join(okt.morphs(sentence))
    sequences = [token_to_idx(tokenized_sentence, UNK_IDX)]
    sequences = pad_sequences(sequences, padding='post', truncating='post', maxlen=MAX_LEN, value=len(w2v_model.wv.index_to_key) + 1)
    return sequences

result = model.predict(sentence_to_sequence('우리 아이가 좋아하는 장난감 세트'))
result

label_list[result.argmax()]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'완구/매트'

In [45]:
def predict(sentence):
    result = model.predict(sentence_to_sequence(sentence))
    return label_list[result.argmax()]
predict('분위기 좋은 스피커')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'음향가전'

In [52]:
def predict(sentence):
    result = model.predict(sentence_to_sequence(sentence))
    return label_list[result.argmax()]
predict('자동차용 엔진오일')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


'자동차용품'

In [54]:
model.save('./category_model/category.h5')

In [56]:
new_model = tf.keras.models.load_model('./category_model/category.h5')
result = new_model.predict(sentence_to_sequence('우리 아이가 좋아하는 용가리치킨'))
result

label_list[result.argmax()]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


'축산물'